In [39]:
import pandas as pd
import tensorflow as tf
data=pd.read_csv("../data/intents.csv")

In [40]:
# Encodage des labels

label_index = pd.factorize(data['Label'])[0]
label_mapping = dict(zip(range(len(data['Label'].unique())), data['Label'].unique()))

data['encoded_Label']=pd.factorize(data['Label'])[0]



In [41]:
label_mapping_json = dict((str(i), label) for i, label in enumerate(data['Label'].unique()))

import json

# Sauvegarde du mapping des labels dans un fichier JSON
with open('../data/label_mapping.json', 'w') as f:
    json.dump(label_mapping_json, f)


In [42]:
corpus = data['Question'].to_list()

labels = data['encoded_Label'].to_list()
#labels = data['Label'].to_list()

In [43]:
# Utiliser len() pour obtenir le nombre d'éléments uniques
nombre_unique_labels = len(set(labels))

In [44]:
nombre_unique_labels

15

In [45]:
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 1

In [46]:
# Séparation train test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(corpus, labels, test_size=0.2, random_state=0, stratify=labels)

In [47]:
X_test

['Can I expect gastronomic innovation in your menu?',
 'Do you offer a selection of cocktails?',
 'Dishes found in Michelin-starred restaurants?',
 "Can you tell me what's on the menu?",
 'Can I have 2 spinach raviolis and 1 glass of red wine?',
 'Can you remove 2 bottles of mineral water from my order?',
 "What's the address of the restaurant?",
 'Do you have a sommelier to pair wines with your gastronomic dishes?',
 'Do you offer free delivery for Adds over a certain amount?',
 'Are your dishes prepared according to halal standards?',
 'Heya',
 "I've decided against 1 spinach ravioli, please remove it.",
 'Dishes containing soy ingredients?',
 'Can I please get 2 duck breasts, 1 glass of red wine, and a bottle of water?',
 'Are you considered a gastronomic hotspot in the area?',
 'Give me a spinach ravioli and a glass of water.',
 'Where is the restaurant situated?',
 'Do you offer halal-certified snacks?',
 'Do I need to make a reservation for brunch?',
 'Hi!',
 'Bread options free 

In [48]:
y_test

[11,
 8,
 12,
 2,
 13,
 14,
 0,
 11,
 5,
 9,
 7,
 14,
 4,
 13,
 11,
 13,
 0,
 9,
 6,
 7,
 4,
 5,
 0,
 7,
 3,
 12,
 14,
 1,
 11,
 3,
 9,
 6,
 1,
 6,
 2,
 13,
 13,
 0,
 10,
 10,
 3,
 5,
 13,
 12,
 5,
 1,
 13,
 8,
 3,
 7,
 4,
 2,
 8,
 1,
 10,
 0,
 14,
 9,
 13,
 14,
 0,
 14,
 12,
 6,
 1,
 4,
 10,
 8]

In [49]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

epochs = 10

# Initialisation du tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Fonction pour tokeniser les données
def data_tokenizer(data, tokenizer):
    return tokenizer(data, padding=True, truncation=True, max_length=512, return_tensors="tf")

# Préparation et tokenisation des données d'entraînement et de test
tokenized_X_train = data_tokenizer(X_train, tokenizer)
tokenized_X_test = data_tokenizer(X_test, tokenizer)

# Conversion des données tokenisées en dataset TensorFlow
train_dataset = tf.data.Dataset.from_tensor_slices((
    {key: val.numpy() for key, val in tokenized_X_train.items()},
    y_train
)).shuffle(1000).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {key: val.numpy() for key, val in tokenized_X_test.items()},
    y_test
)).batch(16)

# Chargement du modèle
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=nombre_unique_labels)

# Compilation du modèle
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Initialisation de la variable pour conserver le meilleur modèle
best_val_accuracy = 0.0
best_model = None

# Entraînement du modèle avec une boucle pour vérifier la précision de validation
for epoch in range(epochs):  # Remplacer par le nombre d'époques désiré
    print(f'Epoch {epoch + 1}/{epochs}')
    history = model.fit(train_dataset, epochs=1, validation_data=test_dataset)  # Entraînement pour une époque à chaque fois

    # Obtention de la précision de validation pour l'époque actuelle
    val_accuracy = history.history['val_accuracy'][0]
    
    # Vérification si le modèle actuel est le meilleur
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        # Sauvegarde du modèle entier ou des poids, selon les besoins
        best_model = model.get_config()  # Pour l'architecture du modèle
        best_weights = model.get_weights()  # Pour les poids du modèle

# Vous pouvez maintenant réutiliser `best_model` et `best_weights` comme vous le souhaitez



All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
17/17 [==============================] - 38s 1s/step - loss: 2.5154 - accuracy: 0.2390 - val_loss: 2.1494 - val_accuracy: 0.4265
Epoch 2/10
17/17 [==============================] - 17s 1s/step - loss: 1.9376 - accuracy: 0.6176 - val_loss: 1.5893 - val_accuracy: 0.6765
Epoch 3/10
17/17 [==============================] - 17s 1s/step - loss: 1.3702 - accuracy: 0.8566 - val_loss: 1.0563 - val_accuracy: 0.8676
Epoch 4/10
17/17 [==============================] - 17s 1s/step - loss: 0.8544 - accuracy: 0.9926 - val_loss: 0.6600 - val_accuracy: 0.9412
Epoch 5/10
17/17 [==============================] - 17s 1s/step - loss: 0.4958 - accuracy: 0.9963 - val_loss: 0.4079 - val_accuracy: 0.9412
Epoch 6/10
17/17 [==============================] - 18s 1s/step - loss: 0.2982 - accuracy: 0.9963 - val_loss: 0.3260 - val_accuracy: 0.9559
Epoch 7/10
17/17 [==============================] - 17s 1s/step - loss: 0.1864 - accuracy: 1.0000 - val_loss: 0.2276 - val_accuracy: 0.9706
Epoch 8/10
17/17 [==

In [50]:
model.set_weights(best_weights)

In [51]:
model.save_weights('../models/model_weights')

In [52]:
# Évaluation du modèle sur l'ensemble de test
results = model.evaluate(test_dataset)
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")


5/5 [==============================] - 1s 163ms/step - loss: 0.2276 - accuracy: 0.9706
Test Loss: 0.227578803896904, Test Accuracy: 0.970588207244873


In [53]:
# Évaluation du modèle sur l'ensemble de test classe par classe
from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(test_dataset)
y_pred = np.argmax(y_pred.logits, axis=1)

print(classification_report(y_test, y_pred, target_names=label_mapping.values()))


5/5 [==============================] - 3s 147ms/step
               precision    recall  f1-score   support

     Location       1.00      1.00      1.00         6
Opening_Hours       1.00      1.00      1.00         5
         Menu       1.00      1.00      1.00         3
      Payment       1.00      1.00      1.00         4
    Allergens       1.00      0.50      0.67         4
     Delivery       1.00      1.00      1.00         4
  Reservation       1.00      1.00      1.00         4
    Greetings       1.00      1.00      1.00         4
       Alcool       1.00      1.00      1.00         4
        Halal       1.00      1.00      1.00         4
   Vegetarien       0.67      1.00      0.80         4
  Gastronomic       1.00      1.00      1.00         4
       Etoile       1.00      1.00      1.00         4
          Add       1.00      1.00      1.00         8
          Del       1.00      1.00      1.00         6

     accuracy                           0.97        68
    macro 

In [54]:
# Fonction pour préparer les données de prédiction
def prepare_predict_data(data, tokenizer):
    tokenized_data = tokenizer(data, padding=True, truncation=True, max_length=512, return_tensors="tf")
    return {key: val.numpy() for key, val in tokenized_data.items()}

# Nouvelles données à prédire
new_data = ["How can i go to the restaurant ?",
            "What do you have to eat ?",
            "When does the restaurant open ?",
            "How can i pay ?",
            "There is any allergens ?","there is delivery ?",
            "Can i have a table ?",
            "Hi",
            "Do you have alcohol ?",
            "You restaurant is halal ?",
            "Any vegetarian options ?",
            "are you a gastronomic restaurant ?",
            "your restaurant has stars ?",
            "Give me 3 beef tartares and 1 glass of orange juice.",
            "Could you remove the duck breasts from our selections?"]

# Préparation des données pour la prédiction
predict_data = prepare_predict_data(new_data, tokenizer)

# Faire des prédictions
predictions = model.predict(predict_data)

# Décoder les prédictions
predicted_classes = tf.argmax(predictions.logits, axis=1).numpy()

# Traduire les prédictions en labels textuels
predicted_labels = [label_mapping[i] for i in predicted_classes]

# Score de confiance pour chaque prédiction
confidence = tf.nn.softmax(predictions.logits, axis=1).numpy().max(axis=1)

# Afficher les prédictions avec les labels textuels
for i, sentence in enumerate(new_data):
    print(f"Sentence: '{sentence}'\nPredicted class: {predicted_labels[i]}\nConfidence: {confidence[i]:.4f}\n\n")


1/1 [==============================] - 2s 2s/step
Sentence: 'How can i go to the restaurant ?'
Predicted class: Location
Confidence: 0.8492


Sentence: 'What do you have to eat ?'
Predicted class: Menu
Confidence: 0.4036


Sentence: 'When does the restaurant open ?'
Predicted class: Opening_Hours
Confidence: 0.9288


Sentence: 'How can i pay ?'
Predicted class: Payment
Confidence: 0.7240


Sentence: 'There is any allergens ?'
Predicted class: Allergens
Confidence: 0.6640


Sentence: 'there is delivery ?'
Predicted class: Delivery
Confidence: 0.8439


Sentence: 'Can i have a table ?'
Predicted class: Reservation
Confidence: 0.7701


Sentence: 'Hi'
Predicted class: Greetings
Confidence: 0.8482


Sentence: 'Do you have alcohol ?'
Predicted class: Alcool
Confidence: 0.8190


Sentence: 'You restaurant is halal ?'
Predicted class: Halal
Confidence: 0.8762


Sentence: 'Any vegetarian options ?'
Predicted class: Vegetarien
Confidence: 0.8991


Sentence: 'are you a gastronomic restaurant ?'
Pre